In [1]:
###### 최근 영업일 기준 데이터 받기 #####
import requests as rq
from bs4 import BeautifulSoup
import re

url = 'https://finance.naver.com/sise/sise_deposit.naver'
data = rq.get(url)
data_html = BeautifulSoup(data.content, "html.parser")

#똑같은 부분이 여러개 위치할 때는 select_one() 사용할 것 -> 맨 위 데이터만 선택
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah'
).text


#숫자만 뽑아내기
biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

In [ ]:
import json
import requests as rq
import pandas as pd

#dt 부분에 biz_day 
url = f'''https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd=G10'''
#json데이터 받기
data = rq.get(url).json()

#data는 딕셔너리인데 딕셔너리 내부에 각각의 값은 리스트 형태이다.
#이것을 데이터 프레임으로 바꾸겠다.
data_pd = pd.json_normalize(data['list'])
print(data_pd)


In [3]:
#반복문을 통해 모든 섹터 종목 불러오기
import time
import json
import requests as rq
import pandas as pd
from tqdm import tqdm #진행속도 확인

sector_code = [
    'G25', 'G35', 'G50', 'G40', 'G10', 'G20', 'G55', 'G30', 'G15', 'G45' 
]

data_sector = []

for i in tqdm(sector_code):
    url = f'''https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd={i}'''
    data = rq.get(url).json()
    data_pd = pd.json_normalize(data['list'])

    data_sector.append(data_pd)

    time.sleep(2)#무한 크롤링 예방


100%|██████████| 10/10 [00:22<00:00,  2.21s/it]


In [5]:
#데이터 합치기
kor_sector = pd.concat(data_sector, axis = 0)
kor_sector = kor_sector[['IDX_CD', 'CMP_CD', 'CMP_KOR', 'SEC_NM_KOR']]#필요한 열만 선택
kor_sector['기준일'] = biz_day
kor_sector['기준일'] = pd.to_datetime(kor_sector['기준일'])

In [6]:
print(kor_sector)

    IDX_CD  CMP_CD CMP_KOR SEC_NM_KOR        기준일
0      G25  005380     현대차    경기관련소비재 2023-09-04
1      G25  000270      기아    경기관련소비재 2023-09-04
2      G25  012330   현대모비스    경기관련소비재 2023-09-04
3      G25  051900  LG생활건강    경기관련소비재 2023-09-04
4      G25  090430  아모레퍼시픽    경기관련소비재 2023-09-04
..     ...     ...     ...        ...        ...
648    G45  067730    로지시스         IT 2023-09-04
649    G45  065690     파커스         IT 2023-09-04
650    G45  032960    동일기연         IT 2023-09-04
651    G45  067770   세진티에스         IT 2023-09-04
652    G45  033200     모아텍         IT 2023-09-04

[2410 rows x 5 columns]


In [9]:
#mysql로 데이터 옮기기
import pymysql

con = pymysql.connect(user='root',
                       passwd = '8019',
                       host = '127.0.0.1',
                       db = 'stock_db',
                       charset = 'utf8')

mycursor = con.cursor()
query = f"""
    insert into kor_sector (IDX_CD, CMP_CD, CMP_KOR, SEC_NM_KOR, 기준일)
    values (%s, %s, %s, %s, %s) as new
    on duplicate key update
    IDX_CD = new.IDX_CD, CMP_KOR = new.CMP_KOR, SEC_NM_KOR = new.SEC_NM_KOR;
"""
#upsert 쿼리 작성

args = kor_sector.values.tolist()
mycursor.executemany(query, args)
con.commit()

con.close()